### For Restoring the Model

Code for Defining the DataFlow Graph will remain same except there will be no training of data so no optimizing sessions will be there.


In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import glob
import os
import random
import copy

In [2]:
img_height = 200
img_width = 200
num_channels = 3
num_classes = 5
img_size_flat = 200*200*3

filter_size1 = 11
num_filters1 = 32

filter_size2 = 5
num_filters2 = 32

filter_size3 = 2
num_filters3 = 64
#x = tf.placeholder(tf.float32, shape=[None, img_size_flat], name='x')

#x_image = tf.reshape(x, [-1, img_height, img_width, num_channels], name='x_image')

x_image = tf.placeholder(tf.float32, shape=[None, img_height, img_width, num_channels])

y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name= 'y_true')

y_true_cls = tf.argmax(y_true, axis = 1, name='y_true_cls')

weights = {
    'h1': tf.Variable(tf.truncated_normal([filter_size1, filter_size1, num_channels, num_filters1], stddev=0.05)),
    'h2': tf.Variable(tf.truncated_normal([filter_size2, filter_size2, 32, num_filters2], stddev=0.05)),
    'newh1': tf.Variable(tf.truncated_normal([filter_size3, filter_size3, 32, num_filters3], stddev=0.05)),
    'h3': tf.Variable(tf.truncated_normal([7*7*64, 7*7*64],stddev=0.05)),
    'newh2': tf.Variable(tf.truncated_normal([7*7*64, 7*7*64],stddev=0.05)),
    'newh3': tf.Variable(tf.truncated_normal([7*7*64, 1024],stddev=0.05)),
    'out': tf.Variable(tf.truncated_normal([1024, 5],stddev=0.05))

}
biases = {
    'b1': tf.Variable(tf.constant(0.05, shape=[num_filters1])),
    'b2': tf.Variable(tf.constant(0.05, shape=[num_filters2])),
    'newb1': tf.Variable(tf.constant(0.05, shape=[num_filters3])),
    'b3': tf.Variable(tf.constant(0.05, shape=[7*7*64])),
    'newb2': tf.Variable(tf.constant(0.05, shape=[7*7*64])),
    'newb3': tf.Variable(tf.constant(0.05, shape=[1024])),
    'out': tf.Variable(tf.constant(0.05,shape=[5]))

}

In [3]:
conv1 = tf.nn.conv2d(input=x_image, filter= weights['h1'], strides=[1,4,4,1], padding='SAME') 
bias_add1 = tf.nn.bias_add(conv1, biases['b1'])
relu1 = tf.nn.relu(bias_add1)
pool1 = tf.nn.max_pool(value=relu1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(conv1)
print(pool1)

Tensor("Conv2D:0", shape=(?, 50, 50, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 25, 25, 32), dtype=float32)


In [4]:
conv2 = tf.nn.conv2d(input=pool1, filter= weights['h2'], strides=[1,1,1,1], padding='SAME') 
bias_add2 = tf.nn.bias_add(conv2, biases['b2'])
pool2 = tf.nn.max_pool(value=bias_add2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
relu2 = tf.nn.relu(pool2)
print(conv2)
print(pool2)

Tensor("Conv2D_1:0", shape=(?, 25, 25, 32), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 13, 13, 32), dtype=float32)


In [5]:
conv3 = tf.nn.conv2d(input=relu2, filter= weights['newh1'], strides=[1,1,1,1], padding='SAME') 
bias_add3 = tf.nn.bias_add(conv3, biases['newb1'])
pool3 = tf.nn.max_pool(value=bias_add3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
relu3 = tf.nn.relu(pool3)
print(conv3)
print(pool3)
print(relu3)

Tensor("Conv2D_2:0", shape=(?, 13, 13, 64), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Relu_2:0", shape=(?, 7, 7, 64), dtype=float32)


In [6]:
pool2_flat = tf.reshape(relu3, [-1, 7*7*64])
pool2_flat

<tf.Tensor 'Reshape:0' shape=(?, 3136) dtype=float32>

In [7]:
fc_layer = tf.add(tf.matmul(pool2_flat, weights['h3']), biases['b3'], name='fc_layer')
relu4 = tf.nn.relu(fc_layer)
relu4

<tf.Tensor 'Relu_3:0' shape=(?, 3136) dtype=float32>

In [8]:
fc_layer_1 = tf.add(tf.matmul(relu4, weights['newh2']), biases['newb2'], name='fc_layer_1')
relu5 = tf.nn.relu(fc_layer_1)

In [9]:
fc_layer_l = tf.add(tf.matmul(relu5, weights['newh3']), biases['newb3'], name='fc_layer_l')
relu6 = tf.nn.relu(fc_layer_l)
relu6

<tf.Tensor 'Relu_5:0' shape=(?, 1024) dtype=float32>

In [10]:
dropout = tf.nn.dropout(relu6, 0.25)
dropout

<tf.Tensor 'dropout/mul:0' shape=(?, 1024) dtype=float32>

In [11]:
fc_layer2 = tf.add(tf.matmul(dropout, weights['out']), biases['out'], name='fc_layer2')

In [12]:
y_pred_cls = tf.argmax(fc_layer2, axis=1, name='y_pred_cls')
y_pred_prbl = tf.nn.softmax(fc_layer2, name='y_pred_prbl')

In [13]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc_layer2,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy, name='cost')

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3, name='optimizer').minimize(cost)

In [15]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')

### Code for Defining DataFlow Graph ends here.
### Now we will restore the Trained Weights



## Restoring Trained Model 

Weights for the following Variables are now restored for testing:

1. weights 
2. biases

Trained Weights for these variables will be used to test the data.

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

model_path = "Jack Class new/JC"
saver = tf.train.Saver()
saver.restore(sess, model_path)

## Importing Test Data 

Test Data Contains 50 (200x200) sized images for each class.

Total Number of Test Data = 250

In [17]:
classes = ['Jackets', 'Jeans', 'Skirts', 'Sunglasses', 'Tops']

cwd = os.getcwd()
list_files = os.listdir('/home/shresth/tust/Jeans/')

y_true_batch = []
for nm in list_files:
    for cl in classes:
            if cl in nm:
                one_hot = [0., 0., 0., 0., 0.]
                one_hot[classes.index(cl)] = 1.
                y_true_batch.append(one_hot)
y_true_batch = np.array(y_true_batch)

test = [cv2.imread('/home/shresth/tust/Jeans/' + i) for i in list_files]
test = np.array(test)
print(test.shape)
print(y_true_batch.shape)
#list_files[0]
print(list_files[0])
print(y_true_batch[0])

(250, 200, 200, 3)
(250, 5)
33_Skirts_resized.jpg
[0. 0. 1. 0. 0.]


## Running Tensorflow DataFlow Graph

85.6% accuracy is obtained from the testing data

In [18]:
res = sess.run(accuracy, feed_dict={x_image: test, y_true: y_true_batch})
res

0.856

In [19]:
sess.close()